# Index 객체, reindex

- 표 형식의 데이터에서 각 행과 열에 대한 헤더(이름)과 다른 메타데이터(축의 이름)를 저장하는 객체
- Series나 DataFrame 객체를 생성할 때 사용되는 배열이나 또는 순차적인 이름은 내부적으로 색인으로 변환됨

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

### 1. Index 객체

In [3]:
obj = Series(range(3), index=['a', 'b', 'c'])
obj

a    0
b    1
c    2
dtype: int64

In [4]:
idx = obj.index
idx

Index(['a', 'b', 'c'], dtype='object')

In [6]:
idx[1]

'b'

In [7]:
idx[1:]

Index(['b', 'c'], dtype='object')

### 2. ReIndex

- 재색인(ReIndex): 새로운 색인에 맞도록 객체를 새로 생성하는 기능, row, column index 모두 변경 가능
- 주의: Index 객체는 변경할 수 없음, 변경을 위해서는 reindex 사용

In [4]:
#idx[1] = 'd'
#idx[1]
#TypeError: Index does not support mutable operations

- Index객체는 변경 불가능하나 미리 만들어서 공유할 수 있음

In [8]:
idx2 = pd.Index(np.arange(3))
idx2

Int64Index([0, 1, 2], dtype='int64')

#### 1) Series 객체의 reindex

In [9]:
obj2 = Series([2.3, 4.3, -4.1, 3.5], index=['d', 'b', 'a', 'c'])
obj2

d    2.3
b    4.3
a   -4.1
c    3.5
dtype: float64

In [10]:
obj2 = obj2.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -4.1
b    4.3
c    3.5
d    2.3
e    NaN
dtype: float64

In [11]:
obj3 = obj2.reindex(['a', 'b', 'c', 'd', 'e', 'f'], fill_value=0.0)
obj3

a   -4.1
b    4.3
c    3.5
d    2.3
e    NaN
f    0.0
dtype: float64

#### 2) DataFrame reindex

In [12]:
df = DataFrame(np.arange(9).reshape(3,3), 
               index=['a', 'b', 'd'], columns=['x', 'y', 'z'])
df

,x,y,z
a,0,1,2
b,3,4,5
d,6,7,8


In [13]:
df2 = df.reindex(['a', 'b', 'c', 'd'])
df2

,x,y,z
a,0.0,1.0,2.0
b,3.0,4.0,5.0
c,NaN,NaN,NaN
d,6.0,7.0,8.0


In [14]:
col = ['x', 'y', 'w', 'z']
df2 = df.reindex(columns=col)
df2

,x,y,w,z
a,0,1,NaN,2
b,3,4,NaN,5
d,6,7,NaN,8


- 보간법: 주로 시계열 데이터에서 값을 보간하기 위해 사용됨
- method='ffill'은 앞에 있는 값으로 보간
- DataFrame 객체에서 보간: row에 대해서만 보간

In [16]:
df3 = df.reindex(index=['a', 'b', 'c', 'd'], columns=col)
df3

,x,y,w,z
a,0.0,1.0,NaN,2.0
b,3.0,4.0,NaN,5.0
c,NaN,NaN,NaN,NaN
d,6.0,7.0,NaN,8.0


In [17]:
df3 = df.reindex(index=['a', 'b', 'c', 'd'], columns=col, method='ffill')
df3

,x,y,w,z
a,0,1,NaN,2
b,3,4,NaN,5
c,3,4,NaN,5
d,6,7,NaN,8


#### 3) ix를 이용한 reindex

In [18]:
df

,x,y,z
a,0,1,2
b,3,4,5
d,6,7,8


In [20]:
df4 = df.ix[['a', 'b', 'c', 'd'], col]
df4

,x,y,w,z
a,0.0,1.0,NaN,2.0
b,3.0,4.0,NaN,5.0
c,NaN,NaN,NaN,NaN
d,6.0,7.0,NaN,8.0


### 3. Row, Colum 삭제

#### 1) Series에서 삭제하기

In [21]:
obj4 = Series(np.arange(5), index=['a', 'b', 'c', 'd', 'e'])
obj4

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [22]:
obj5 = obj4.drop('c')
obj5

a    0
b    1
d    3
e    4
dtype: int64

In [23]:
obj5 = obj4.drop(['b', 'd', 'c'])
obj5

a    0
e    4
dtype: int64

#### 2) DataFrame에서 row 삭제하기

In [24]:
df5 = DataFrame(np.arange(16).reshape(4,4),
               index=['seoul', 'busan', 'daegu', 'incheon'],
               columns=['one', 'two', 'three', 'four'])
df5

,one,two,three,four
seoul,0,1,2,3
busan,4,5,6,7
daegu,8,9,10,11
incheon,12,13,14,15


In [25]:
new_df = df5.drop(['seoul', 'busan'])
new_df

,one,two,three,four
daegu,8,9,10,11
incheon,12,13,14,15


#### 3) DataFrame에서 colum 삭제하기

- column과 axis를 같이 줘야함

In [26]:
new_df = df5.drop('three', axis=1)
new_df

,one,two,four
seoul,0,1,3
busan,4,5,7
daegu,8,9,11
incheon,12,13,15


In [27]:
new_df = df5.drop(['two', 'four'], axis=1)
new_df

,one,three
seoul,0,2
busan,4,6
daegu,8,10
incheon,12,14
